In [1]:


%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import os
import sys
sys.path.append("../")
import random
from aif360.algorithms.inprocessing.gerryfair_classifier import *
from aif360.algorithms.inprocessing.gerryfair.clean import *
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import *
from aif360.algorithms.inprocessing.gerryfair.auditor import *
from sklearn import svm
from sklearn import tree
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
from IPython.display import Image
import pickle
import matplotlib.pyplot as plt



We first demonstrate how to instantiate a `Model`, `train` it with respect to rich subgroup fairness, and `predict` the label of a new example. We remark that when we set the `print_flag = True` at each iteration of the algorithm we print the error, fairness violation, and violated group size of most recent model. The error is the classification error of the classifier. At each round the Learner tries to find a classifier that minimizes the classification error plus a weighted sum of the fairness disparities on all the groups that the Auditor has found up until that point. By contrast the Auditor tries to find the group at each round with the greatest rich subgroup disparity with respect to the Learner's model. We define `violated group size` as the size (as a fraction of the dataset size) of this group, and the `fairness violation` as the `violated group size` times the difference in the statistical rate (FP or FN rate) on the group vs. the whole population. 

In the example below we set `max_iterations=5000` which is in-line with time to convergence observed in [the rich subgroup fairness empirical paper](https://arxiv.org/abs/1808.08166), but advise that this can be highly dataset dependent. Our target $\gamma$-disparity is `gamma = .001`, our statistical rate is false positive rate or `FP`, and our weighted regression oracle is linear regression. We observe that the unconstrained (with no fairness constraint) classifier has error $.185$ and $\gamma$-disparity $.022$. After $5000$ iterations we obtain a classifier that $\gamma$-fair, and has error $.230$. We note that we converge to a fair, and almost as accurate classifier after just a hundred iterations.


In [ ]:
C = 10
print_flag = True
gamma = .001
max_iterations = 2000

fair_model = Model(C=C, printflag=print_flag, gamma=gamma, fairness_def='FP',
             max_iters=max_iterations, heatmapflag=False)
data_set = load_preproc_data_adult(sub_samp=200, balance=True)

# fit method
communities_all_errors, communities_violations = fair_model.fit(data_set,
                                                 early_termination=True, return_values=True)

# predict method. If threshold in (0, 1) produces binary predictions

dataset_yhat = fair_model.predict(data_set, threshold=False)




iteration: 1, error: 0.2, fairness violation: 0.043500000000000004, violated group size: 0.225
iteration: 2, error: 0.2975, fairness violation: 0.022875000000000003, violated group size: 0.225
iteration: 3, error: 0.33000000000000007, fairness violation: 0.016, violated group size: 0.225
iteration: 4, error: 0.34625, fairness violation: 0.012562500000000002, violated group size: 0.275
iteration: 5, error: 0.3559999999999999, fairness violation: 0.010500000000000002, violated group size: 0.225
iteration: 6, error: 0.36249999999999993, fairness violation: 0.009125000000000006, violated group size: 0.275
iteration: 7, error: 0.3671428571428571, fairness violation: 0.008142857142857148, violated group size: 0.275
iteration: 8, error: 0.370625, fairness violation: 0.00740625, violated group size: 0.225
iteration: 9, error: 0.3733333333333334, fairness violation: 0.006833333333333336, violated group size: 0.275
iteration: 10, error: 0.37550000000000006, fairness violation: 0.0063750000000000

iteration: 77, error: 0.3924675324675324, fairness violation: 0.0027857142857142855, violated group size: 0.275
iteration: 78, error: 0.3925, fairness violation: 0.002778846153846154, violated group size: 0.275
iteration: 79, error: 0.3925316455696203, fairness violation: 0.002772151898734178, violated group size: 0.225
iteration: 80, error: 0.39256250000000004, fairness violation: 0.0027656250000000003, violated group size: 0.275
iteration: 81, error: 0.3925925925925926, fairness violation: 0.0027592592592592603, violated group size: 0.275
iteration: 82, error: 0.3926219512195123, fairness violation: 0.0027530487804878056, violated group size: 0.225
iteration: 83, error: 0.39265060240963856, fairness violation: 0.0027469879518072287, violated group size: 0.225
iteration: 84, error: 0.39267857142857143, fairness violation: 0.002741071428571429, violated group size: 0.275
iteration: 85, error: 0.3927058823529411, fairness violation: 0.0027352941176470597, violated group size: 0.275
iter

iteration: 152, error: 0.39371710526315795, fairness violation: 0.0025213815789473697, violated group size: 0.275
iteration: 153, error: 0.39372549019607844, fairness violation: 0.002519607843137256, violated group size: 0.275
iteration: 154, error: 0.3937337662337662, fairness violation: 0.002517857142857144, violated group size: 0.275
iteration: 155, error: 0.393741935483871, fairness violation: 0.002516129032258064, violated group size: 0.275
iteration: 156, error: 0.39375, fairness violation: 0.0025144230769230777, violated group size: 0.275
iteration: 157, error: 0.39375796178343947, fairness violation: 0.002512738853503185, violated group size: 0.225
iteration: 158, error: 0.39376582278481004, fairness violation: 0.002511075949367089, violated group size: 0.275
iteration: 159, error: 0.3937735849056604, fairness violation: 0.002509433962264151, violated group size: 0.225
iteration: 160, error: 0.39378125, fairness violation: 0.002507812500000001, violated group size: 0.275
iterat

In [ ]:

# output heatmap (brute force)
# replace None with the relative path if you want to save the plot
fair_model.heatmapflag = True
fair_model.heatmap_path  = 'heatmap'
fair_model.generate_heatmap(data_set, dataset_yhat.labels)
Image(filename='{}.png'.format(fair_model.heatmap_path)) 





In [ ]:
# auditing a classifier for unfairness
# instantiate auditor
auditor = Auditor(data_set, 'FP')
metric_baseline = auditor.get_baseline(array_to_tuple(data_set.labels), array_to_tuple(dataset_yhat.labels))
group = auditor.get_group(dataset_yhat.labels, metric_baseline)
print('gamma disparity: {}'.format(group.weighted_disparity))






In [ ]:
def multiple_classifiers_pareto(dataset, gamma_list=[0.002, 0.005, 0.01, 0.02, 0.05, 0.1], save_results=False, iters=1000):

    ln_predictor = linear_model.LinearRegression()
    svm_predictor = svm.LinearSVR()
    tree_predictor = tree.DecisionTreeRegressor(max_depth=3)
    kernel_predictor = KernelRidge(alpha=1.0, gamma=1.0, kernel='rbf')
    predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': iters},
                      'SVR': {'predictor': svm_predictor, 'iters': iters},
                      'Tree': {'predictor': tree_predictor, 'iters': iters},
                      'Kernel': {'predictor': kernel_predictor, 'iters': iters}}

    results_dict = {}

    for pred in predictor_dict:
        print('Curr Predictor: {}'.format(pred))
        predictor = predictor_dict[pred]['predictor']
        max_iters = predictor_dict[pred]['iters']
        fair_clf = Model(C=100, printflag=True, gamma=1, predictor=predictor, max_iters=max_iters)
        fair_clf.printflag = False
        fair_clf.set_options(max_iters=max_iters)
        errors, fp_violations, fn_violations = fair_clf.pareto(dataset, gamma_list)
        results_dict[pred] = {'errors': errors, 'fp_violations': fp_violations, 'fn_violations': fn_violations}
        plt.plot(errors, fp_violations, label=pred)

    if save_results:
        pickle.dump(results_dict, open('results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb'))

    plt.xlabel('Error')
    plt.ylabel('Unfairness')
    plt.legend()
    plt.title('Error vs. Unfairness\n(Communities & Crime Dataset)')
    plt.show()

multiple_classifiers_pareto(data_set)

In [ ]:

def fp_vs_fn(dataset, gamma_list, iters):
    fp_auditor = Auditor(dataset, 'FP')
    fn_auditor = Auditor(dataset, 'FN')
    fp_violations = []
    fn_violations = []
    for g in gamma_list:
        fair_model = Model(C=100, printflag=False, gamma=g, max_iters=iters)
        fair_model.set_options(gamma=g)
        fair_model.fit(dataset)
        predictions = (fair_model.predict(dataset)).labels
        predictions_inv = tuple([1 - p for p in predictions])
        _, fp_diff = fp_auditor.audit(predictions)
        _, fn_diff = fn_auditor.audit(predictions_inv)
        fp_violations.append(fp_diff)
        fn_violations.append(fn_diff)

    print((fp_violations, fn_violations))

    plt.plot(fp_violations, fn_violations, label='communities')
    plt.xlabel('False Positive Disparity')
    plt.ylabel('False Negative Disparity')
    plt.legend()
    plt.title('FP vs FN Unfairness')
    plt.show()
    

gamma_list = [0.001, 0.002, 0.003, 0.004, 0.005, 0.0075, 0.01, 0.02, 0.03, 0.05]
fp_vs_fn(data_set, gamma_list, 5)